In [1]:
import pandas as pd
import numpy as np
import os
from datetime import datetime

In [2]:
"""
Input: p10 combined pba40 plu and basis boc data, with 
Output: p10 combined pba40 plu and basis boc data, with: 
    1) NaN in basis allowed development types filled in using pba40 data
    2) basis allowed development types replaced by pba40 data following the 'hybrid index'
"""

if os.getenv('USERNAME')=='ywang':
    # 'data_folder' is the output folder of the script '1_PLU_BOC_data_combine'
    data_folder  = 'C:\\Users\\ywang\\Documents\\Files_for_Py\\BASIS\\BOC\\outputs'
    idx_folder  = 'C:\\Users\\ywang\\Documents\\Files_for_Py\\UrbanSim_input_Zoning\\outputs\\hybrid'
    
ALLOWED_BUILDING_TYPE_CODES = ["HS","HT","HM","OF","HO","SC","IL","IW","IH","RS","RB","MR","MT","ME"]

today = datetime.today().strftime('%Y_%m_%d')

In [3]:
## P10 parcels with pba40 plu and basis boc data
plu_boc_file = os.path.join(data_folder, today+'_p10_plu_boc_allAttrs.csv')
plu_boc = pd.read_csv(plu_boc_file)
plu_boc.head()

,PARCEL_ID,COUNTY_ID,CTY_NAME,ACRES,juris_id_zmod,max_far_basis,max_far_pba40,max_dua_basis,max_dua_pba40,max_height_basis,...,RS_basis,RS_pba40,RB_basis,RB_pba40,MR_basis,MR_pba40,MT_basis,MT_pba40,ME_basis,ME_pba40
0,229116.0,1,Alameda,3.360520,livr,0.00,NaN,0.00000,2.00000,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,244166.0,1,Alameda,1.294423,livr,0.35,NaN,14.00000,3.00000,35.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,202378.0,1,Alameda,14.993605,hayw,0.00,NaN,0.00000,8.70000,NaN,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
3,2004420.0,97,Sonoma,316.247146,uson,0.00,NaN,0.01666,0.00417,35.0,...,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0
4,340332.0,1,Alameda,0.621275,frem,0.01,NaN,1.00000,23.00000,30.0,...,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0


In [4]:
## Allowed development type evaluation

for devType in ALLOWED_BUILDING_TYPE_CODES:
    plu_boc[devType+'_comp'] = np.nan

    plu_boc.loc[(plu_boc[devType + '_pba40'] == 1) & 
                (plu_boc[devType + '_basis'] == 0),devType+'_comp'] = 'only PBA40 allow'
    plu_boc.loc[(plu_boc[devType + '_pba40'] == 0) & 
                (plu_boc[devType + '_basis'] == 1),devType+'_comp'] = 'only BASIS allow'
    plu_boc.loc[(plu_boc[devType + '_pba40'] == 1) & 
                (plu_boc[devType + '_basis'] == 1),devType+'_comp'] = 'both allow'
    plu_boc.loc[(plu_boc[devType + '_pba40'] == 0) & 
                (plu_boc[devType + '_basis'] == 0),devType+'_comp'] = 'both not allow'
    plu_boc.loc[plu_boc[devType + '_basis'].isnull(),devType+'_comp'] = 'missing BASIS data'    
    plu_boc.loc[plu_boc['nodev_zmod'    ] == 1,devType+'_comp'] = 'not developable'
            
devType_comp = plu_boc[['PARCEL_ID','COUNTY_ID', 'CTY_NAME', 'ACRES','juris_id_zmod'] + 
                       [devType+'_comp' for devType in ALLOWED_BUILDING_TYPE_CODES]]

devType_comp.to_csv(os.path.join(data_folder, today+'_devType_comparison.csv'),index = False)

In [5]:
## Fill in missing allowed development types in BASIS

def countMissing(df):
    for btype in ALLOWED_BUILDING_TYPE_CODES:
        null_btype_count = len(df.loc[df["{}_basis".format(btype)].isnull()])
        print('Number of parcels missing allowable type for {}: {:,} or {:.1f}%'.format(btype,
           null_btype_count, 100.0*null_btype_count/len(df)))

countMissing(plu_boc)

plu_boc_filled_devTypeNa = plu_boc.copy()

for btype in ALLOWED_BUILDING_TYPE_CODES:
    plu_boc_filled_devTypeNa[btype+'_idx'] = 'BASIS'
    missing_idx = (plu_boc_filled_devTypeNa[btype+'_basis'].isnull()) & (plu_boc_filled_devTypeNa['nodev_zmod'] == 0)
    plu_boc_filled_devTypeNa.loc[missing_idx, btype+'_basis'] = plu_boc_filled_devTypeNa.loc[missing_idx, btype+'_pba40']
    plu_boc_filled_devTypeNa.loc[missing_idx, btype+'_idx'] = 'PBA40_fill_na'

print('After filling nan in BASIS allowed development type using PBA40 data \n')
countMissing(plu_boc_filled_devTypeNa)

plu_boc_filled_devTypeNa.to_csv(os.path.join(data_folder, today+'_p10_plu_boc_fill_naType.csv'),index = False)

#for i in ALLOWED_BUILDING_TYPE_CODES:
#    print(plu_boc_filled_devTypeNa[i+'_idx'].unique())

Number of parcels missing allowable type for HS: 227,036 or 11.6%
Number of parcels missing allowable type for HT: 227,051 or 11.6%
Number of parcels missing allowable type for HM: 233,111 or 11.9%
Number of parcels missing allowable type for OF: 233,011 or 11.9%
Number of parcels missing allowable type for HO: 233,111 or 11.9%
Number of parcels missing allowable type for SC: 233,085 or 11.9%
Number of parcels missing allowable type for IL: 227,839 or 11.6%
Number of parcels missing allowable type for IW: 233,093 or 11.9%
Number of parcels missing allowable type for IH: 233,054 or 11.9%
Number of parcels missing allowable type for RS: 233,111 or 11.9%
Number of parcels missing allowable type for RB: 233,607 or 11.9%
Number of parcels missing allowable type for MR: 233,111 or 11.9%
Number of parcels missing allowable type for MT: 233,111 or 11.9%
Number of parcels missing allowable type for ME: 233,173 or 11.9%
After filling nan in BASIS allowed development type using PBA40 data 

Numbe

In [ ]:
## Index 1: use PBA40 allowed development type and BASIS development intensity for all jurisdictions

idx_BASIS_intensity_all_file = os.path.join(idx_folder, 'idx_BASIS_intensity_all.csv')
idx_BASIS_intensity_all = pd.read_csv(idx_BASIS_intensity_all_file)
idx_BASIS_intensity_all.set_index('juris_id',inplace = True)

plu_boc_pba40Type = plu_boc_filled_devTypeNa.copy()

for juris in list(idx_BASIS_intensity_all.index):
    for btype in ALLOWED_BUILDING_TYPE_CODES:
        if idx_BASIS_intensity_all.loc[juris, btype+'_idx'] == 'PBA40':
            replace_idx = (plu_boc_pba40Type.juris_id_zmod == juris) & (plu_boc_pba40Type[btype+'_idx'] != 'PBA40_fill_na')
            plu_boc_pba40Type.loc[replace_idx, btype+'_basis'] = plu_boc_pba40Type.loc[replace_idx, btype+'_pba40']
            plu_boc_pba40Type.loc[replace_idx, btype+'_idx'] = 'PBA40'

plu_boc_pba40Type.to_csv(os.path.join(data_folder, today+'_p10_plu_boc_pba40Type.csv'),index = False)

In [ ]:
## Index 2: use PBA40 allowed development type for some jurisdictions



In [ ]:
## Index 2: use PBA40 development intensity for some jurisdictions

